This notebook demonstrates the functionality of the new Neptune Client prototype.

Import the prototype as follows:

In [1]:
from neptune_client_prototype import Experiment, Atom, Set, Series, variable

Recall that Neptune is a machine learning experiment tracking tool. A machine learning experiment is an instance of training a model with given hyperparameters and training dataset, and evaluating the trained model on a test dataset. While Neptune provides integrations with various machine learning frameworks, Python's Neptune client in its bare form is a unopinionated logger which logs structured data to a remote server. In practice, logged data would be: the experiment's hyperparameters, metadata, metrics across batches and epochs, etc. 

Below is an example of logging data to Neptune:

In [2]:
exp1 = Experiment('NPT-1')
exp1['training/batch/accuracy'].log(.95)
exp1['training/batch/loss'].log(422.345)
exp1['training/batch/accuracy'].log(.96)
exp1['training/batch/loss'].log(344.344)
exp1['training/epoch/accuracy'].log(.94)
exp1['training/epoch/loss'].log(234.566)

Experiment NPT-1: training/batch/accuracy: log step=0, timestamp=1594046278.527955, value=0.95
Experiment NPT-1: training/batch/loss: log step=0, timestamp=1594046278.533241, value=422.345
Experiment NPT-1: training/batch/accuracy: log step=1, timestamp=1594046278.533371, value=0.96
Experiment NPT-1: training/batch/loss: log step=1, timestamp=1594046278.534185, value=344.344
Experiment NPT-1: training/epoch/accuracy: log step=0, timestamp=1594046278.5342631, value=0.94
Experiment NPT-1: training/epoch/loss: log step=0, timestamp=1594046278.534326, value=234.566


Observe the printed output and note that for each invocation of the `log` method, a line was printed with the experiment's name, path, and the logged value. Fields `step` and `timestamp` will be explained below.

In the production Neptune client, logged data is sent to a server and stored there. This prototype prints logged data to stdout and keep the cumulative state in memory.

If you have used the current release of Neptune, you will be familiar with the notion of viewing experiment data in Neptune UI in the browser. However, it is also possible to retrieve logged experiment data programmatically using Neptune's Python client. The cell below retrieves some of the logged data using methods `tail` and `all`.

In [3]:
training = {'batch': {}, 'epoch': {}}
training['batch']['accuracy'] = exp1['training/batch/accuracy'].tail(1)
training['batch']['loss'] = exp1['training/batch/loss'].tail(2)
training['epoch']['accuracy'] = exp1['training/epoch/accuracy'].all()
training['epoch']['loss'] = exp1['training/epoch/loss'].all()
training

{'batch': {'accuracy': [0.96], 'loss': [422.345, 344.344]},
 'epoch': {'accuracy': [0.94], 'loss': [234.566]}}

Python's Neptune client may be best understood by comparing it with a append-only logger which logs unstructured text. By contrast, Neptune client logs data into *variables*. E.g. consider

`exp1['training/batch/accuracy'].log(.95)`

The value `.95` is logged into the variable which is identified by its path `training/batch/accuracy`. Methods like `tail` or `all` act on variables: `tail(n)` returns a list of `n` last values logged into the variable, and `all()` returns a list of all values logged into the variable.

A variable is created when a writing method like `log` is used on a path for the first time. An attempt to read from a variable which has not been initialized (recall that initialization happends through writing to a new path) raises an error.

In [4]:
exp1['does-not/exist'].tail(42)

AttributeError: 

In [5]:
# TODO provide a better error message

It is important to understand that the lookup / `__getitem__` syntax on an experiment object evaluates to an `ExperimentView`:

In [6]:
type(exp1['some-path'])

neptune_client_prototype.experiment.ExperimentView

An `ExperimentView` is a simple proxy object which stores a reference to the experiment and the path used in the lookup (`'some-path'` in the above example). The path may or may not correspond to an existing variable. If the path corresponds to an existing variable, then method calls on the `ExperimentView` are delegated to that variable. If the path does not correspond to an existing variable, then calling a writing method like `log` on it creates a new variable, whereas calling a reading method on it raises an error, as is demonstrated above.

## Variable types

Suppose we initialize a variable with the `log` method:

In [7]:
exp1['foo'].log(42.0)

Experiment NPT-1: foo: log step=0, timestamp=1594046278.710868, value=42.0


We can examine the type of that variable:

In [8]:
exp1['foo'].variable_type()

(neptune_client_prototype.variable.Series, float)

A Neptune variable type consists of two parts: the *container type* and the *content type*. Performing `exp1['foo'].log(42.0)` created a variable of container type `Series` and of content type `float`.

The Series container type represents a sequence of values which are typically appended, but can also be modified otherwise. We can obtain the list of initializing / modifying / reading methods on a Series as follows:

In [9]:
print(Series.__doc__)


    Writing / initializing methods: log
    Reading methods: tail, all
    


As stated in the docstring, a `Series` can be created / written to using the `log` method, and read from using `tail` and `all` methods.

We can view a list of container types provided by Neptune by inspecting the documentation for the `variable` module:

In [10]:
print(variable.__doc__)


Defines variables and container types.

Neptune define three container types: Atom, Series, and Set.

Detailed documentation on creating / modifying / reading variables of
a container type, refer to the documentation for the given container type,
e.g.

>>> help(Series)



As stated in the documentation, apart from `Series`, two other container types supported by Neptune are `Atom` and `Set`.

We can now examine the `Atom` container type:

In [11]:
print(Atom.__doc__)


    Modifying / initializing methods: assign
    Reading methods: read
    


We can observe the bevavior of its `assign` and `read` methods:

In [12]:
exp1['experiment-author'].assign('John Doe')
print(exp1['experiment-author'].read())
exp1['experiment-author'].assign('Jane Doe')
print(exp1['experiment-author'].read())


Experiment NPT-1: experiment-author: assign John Doe
John Doe
Experiment NPT-1: experiment-author: assign Jane Doe
Jane Doe


A variable of type `Atom` is created by a call to the `assign` method and can be read from using the `read` method. Subsequent calls to `assign` on an `Atom` variable override the value.

In [13]:
# TODO can subsequent `assign` calls change the content type of the method? likewise, can `reset` on a Set change the content type of the Set?

Finally, we examine the `Set` container type.

In [14]:
print(Set.__doc__)


    Modifying / initializing methods: add
    Writing methods: remove, reset
    Reading methods: get
    


In [15]:
exp1['tags'].add('cnn', 'rnn')
print(exp1['tags'].get())
exp1['tags'].add('transformer')
print(exp1['tags'].get())
exp1['tags'].remove('cnn')
print(exp1['tags'].get())
exp1['tags'].reset('attention')
print(exp1['tags'].get())

Experiment NPT-1: tags: add ('cnn', 'rnn')
{'cnn', 'rnn'}
Experiment NPT-1: tags: add ('transformer',)
{'transformer', 'cnn', 'rnn'}
Experiment NPT-1: tags: remove ('cnn',)
{'transformer', 'rnn'}
Experiment NPT-1: tags: reset ('attention',)
{'attention'}


This concludes the discussion of different types of variables and their methods. In the next section, we will discuss organizing variables into namespaces.

# Organizing variables with namespaces

Recall the example from the previous section:

In [16]:
exp1 = Experiment('NPT-1')
exp1['training/batch/accuracy'].log(.95)
exp1['training/batch/loss'].log(422.345)
exp1['training/batch/accuracy'].log(.96)
exp1['training/batch/loss'].log(344.344)
exp1['training/epoch/accuracy'].log(.94)
exp1['training/epoch/loss'].log(234.566)

Experiment NPT-1: training/batch/accuracy: log step=0, timestamp=1594046278.835893, value=0.95
Experiment NPT-1: training/batch/loss: log step=0, timestamp=1594046278.8366148, value=422.345
Experiment NPT-1: training/batch/accuracy: log step=1, timestamp=1594046278.836745, value=0.96
Experiment NPT-1: training/batch/loss: log step=1, timestamp=1594046278.836885, value=344.344
Experiment NPT-1: training/epoch/accuracy: log step=0, timestamp=1594046278.837058, value=0.94
Experiment NPT-1: training/epoch/loss: log step=0, timestamp=1594046278.837183, value=234.566


In [17]:
exp1['training'].namespace_contents()

{'batch': neptune_client_prototype.experiment.Namespace,
 'epoch': neptune_client_prototype.experiment.Namespace}

An experiment exposes methods for logging data. In the real-world implementation, logged data is sent to a Neptune server; this prototype implementation prints logged data to standard output and stores the state of variables in memory.

The following syntax is of special importance:

In [3]:
exp1['name']

Using the lookup / `__getitem__` syntax on an experiment objects evaluates the an `ExperimentView`. An `ExperimentView` is a simple proxy object which stores a reference to the experiment and the key used in the lookup (`'name'` in the above example).

Suppose we have an `ExperimentView` object holding a reference to experiment `exp` and `key` (possibly created by the syntax `exp[key]`). An `ExperimentView` object provides methods for logging data for the given experiment `exp` under the key `key`.

One of such logging methods is, well, the `log` method.

In [4]:
exp1['accuracy'].log(.95)

Experiment NPT-1: accuracy: log step=0, timestamp=1593615609.156553, value=0.95


Note that the printed output of the above cell contains the experiment name "NPT-1", the key "accuracy", and the logged value 0.95. `step` and `timestamp` values will be explained below.

Let us log more "accuracy" values.

In [5]:
exp1['accuracy'].log(.96)
exp1['accuracy'].log(.97)

Experiment NPT-1: accuracy: log step=1, timestamp=1593615912.633049, value=0.96
Experiment NPT-1: accuracy: log step=2, timestamp=1593615912.6331809, value=0.97


If you have used the current release of Neptune, you will be familiar with the notion of viewing experiment data in Neptune UI in the browser. However, it is also possible to retrieve logged experiment data programmatically using Neptune's Python client. The cell below retrieves the last two values logged under the key "accuracy":

In [6]:
exp['accuracy'].tail(2)

[0.96, 0.97]

Note that nothing was printed to stdout, which means that the `tail` method did not perform a write. Instead, it returned a Python list containing the last two values logged under the key "accuracy".

It is now time to introduce new terminlogy. An experiment and a key defines a `Variable`. A `Variable` has a specific structure, which is determined by the write method used to create it. In this case, the structure of the variable `exp['accuracy']` is `Series`, because it was created with a `log` method.

A `Series` variable represents a list of values. It has several read/write methods, of which this tutorial demonstrates the `log` write method and the `tail` read method. It is important to understand that calling a write method on an `ExperimentView` results in creating a variable of the appropriate type.

Let us go through this sequence of steps again, discussing the behavior at each step.

First we create a new experiment.

In [7]:
exp2 = Experiment('NPT-2')

At this point the newly created experiment `exp2` does not contain any variables.

Then call the `log` method on an `ExperimentView`:

In [8]:
exp2['accuracy'].log(0.95)

Experiment NPT-2: accuracy: log step=0, timestamp=1593617003.777708, value=0.95


Let us analyse this command in detail. The above cell in Python is equivalent to:

```python
ev = exp2['accuracy'] # (1)
ev.log(0.95)          # (2)
```

TBC